In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from __future__ import absolute_import, division, print_function

import re
import os
import sys
import datetime
import itertools
import warnings

import pandas as pd
import pandas_datareader.data as web
import numpy as np

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
#from arch import arch_model

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# remove warnings
import warnings
warnings.filterwarnings('ignore')

# prevent crazy long pandas prints
pd.options.display.max_columns = 16
pd.options.display.max_rows = 16
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(precision=5, suppress=True)

# Display and Plotting
import matplotlib.pyplot as plt
#set_matplotlib_formats('pdf', 'png')
plt.rcParams['savefig.dpi'] = 80
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 16, 8
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.titleweight'] = 'bold'
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['text.usetex'] = False
#plt.rcParams['font.family'] = "serif"
plt.rcParams['font.serif'] = "cm"
plt.rcParams['text.latex.preamble'] = b"\usepackage{subdepth}, \usepackage{type1cm}"

import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})


from ipywidgets import interactive, widgets, RadioButtons, ToggleButtons, Select, FloatSlider, FloatProgress

from IPython.display import set_matplotlib_formats, Image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Data

Every problem starts with data....

Obviously there are a multitude of data sets out there. Thus, we will use two data sets: the stylized "International Airline Passengers" data set as well as a self generated data set of stock prices simply denoted <code>data</code> we generate ourseleves later in this section

## 1.1 Working with data (structures)

The <code>pandas</code> module allows to introduce data management to data. using the <code>read_csv</code> method we string our data into a <code>DataFrame</code> object which allows us to directly access the data and provides us with methods which analyse or transform the data. 

A much better formatting of the data can be established if additional arguments are specified when importing the data set:
<ul>
<li><strong>header</strong>: include the first row as the header for the <code>DataFrame</code> object</li>
<li><strong>index_col</strong>: set the index column of the DataFrame to the first column of the data set ('month')</li>
<li><strong>parse_dates</strong>: automatically parse dates which will index the DataFrame</li>
<li><strong>sep</strong>: specify the symbol which seperates the values and strings in the data set - in this case it is semi-colon</li>
</ul>

In [3]:
# load passenger data set and safe to DataFrame
pas = pd.read_csv('./data/passengers.csv', header=0, index_col=0, parse_dates=True, sep=';')

In [4]:
# print the first 5 rows of the DataFrame
pas.head()

,n_passengers
month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


In [5]:
# print the type of an object
type(pas)

pandas.core.frame.DataFrame

In [6]:
# print the data type of all columns except for the index column
pas.dtypes

n_passengers    int64
dtype: object

In [7]:
# print summary statistics
pas.describe()

,n_passengers
count,144.00000
mean,280.29861
std,119.96632
min,104.00000
25%,180.00000
50%,265.50000
75%,360.50000
max,622.00000


Indexes are very important in the domain pandas data transformations and the applicability of prebuilt analytics methods from other libraries.

In [8]:
# list the index of the DataFrame
pas.index

DatetimeIndex(['1949-01-01', '1949-02-01', '1949-03-01', '1949-04-01',
               '1949-05-01', '1949-06-01', '1949-07-01', '1949-08-01',
               '1949-09-01', '1949-10-01',
               ...
               '1960-03-01', '1960-04-01', '1960-05-01', '1960-06-01',
               '1960-07-01', '1960-08-01', '1960-09-01', '1960-10-01',
               '1960-11-01', '1960-12-01'],
              dtype='datetime64[ns]', name='month', length=144, freq=None)

In [9]:
# save the n_passenger column to a new variable, this becomes a Series object
y = pas['n_passengers']

# or
y = pas.n_passengers

In [10]:
# print first 5 rows
y.head()

month
1949-01-01    112
1949-02-01    118
1949-03-01    132
1949-04-01    129
1949-05-01    121
Name: n_passengers, dtype: int64

In [11]:
# print type of the ts object
type(y)

pandas.core.series.Series

In [12]:
# select all entries from the year '1950'
y['1950']

month
1950-01-01    115
1950-02-01    126
1950-03-01    141
1950-04-01    135
1950-05-01    125
1950-06-01    149
1950-07-01    170
1950-08-01    170
1950-09-01    158
1950-10-01    133
1950-11-01    114
1950-12-01    140
Name: n_passengers, dtype: int64

## 1.2 Download data with pandas_datareader

This is a sample workflow of how to retrieve data directly from the web using the <code>pandas_datareader</code> module to access <strong>stock closing price data</strong> from the Yahoo! Finance or Google Finance APIs. Yahoo! Finance is up to now the most popular service, and thus it will also be used here.

In [13]:
# define arguments
tickers = ['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', '^GSPC']
provider = 'yahoo'
start = datetime.datetime(2012, 5, 18)
end = datetime.date(2017, 8, 22)

In [14]:
# ask Yahoo! Finance for data
panel = web.DataReader(tickers, provider, start, end)

While we are at it we can also donwload data about <strong>corporate actions</strong> like stock split or dividend payments

In [15]:
# ask Yahoo! Finance for data
corporate_actions = web.DataReader(tickers, 'yahoo-actions', start, end)

In [16]:
corporate_actions

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 25 (major_axis) x 6 (minor_axis)
Items axis: Stock Splits to Dividends
Major_axis axis: 2012-08-09 00:00:00 to 2017-08-10 00:00:00
Minor_axis axis: AAPL to ^GSPC

In [19]:
# show corporate actions axes
corporate_actions.axes

[Index(['Stock Splits', 'Dividends'], dtype='object'),
 DatetimeIndex(['2012-08-09', '2012-11-07', '2013-02-07', '2013-05-09',
                '2013-08-08', '2013-11-06', '2014-02-06', '2014-03-27',
                '2014-05-08', '2014-06-09', '2014-08-07', '2014-11-06',
                '2015-02-05', '2015-04-27', '2015-05-07', '2015-07-15',
                '2015-08-06', '2015-11-05', '2016-02-04', '2016-05-05',
                '2016-08-04', '2016-11-03', '2017-02-09', '2017-05-11',
                '2017-08-10'],
               dtype='datetime64[ns]', name='Date', freq=None),
 Index(['AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', '^GSPC'], dtype='object')]

In [31]:
corporate_actions.loc['Stock Splits', '2012' ,'AAPL']

Date
2012-08-09    NaN
2012-11-07    NaN
Name: AAPL, dtype: object

A panel can be thought of as a collection od <code>DateFrame</code> object. We are interest in adjusted prices only, that is, prices that are corrected for dividend payments such that we do not exhibit jumps or breaks in the stock time series. Using the <code>loc<7code> selector we can select a column element across all included dataframes

In [ ]:
# get adjusted closing prices from all data series
# this reduces the Panel to a DataFrame
adj_close = panel.loc['Adj Close']

# get all weekdays between 18.05.2012 and 22.08.2017
all_weekdays = pd.date_range(start=start, end=end, freq='B')

# adjust the DataFrame index to be only the weekdays
adj_close = adj_close.reindex(all_weekdays)

# after reindexing we have to amend for missing values
adj_close = adj_close.fillna(method='ffill')

In [ ]:
# have a look at what we have created
adj_close.head()

In [ ]:
type(adj_close)

## 1.3 Build a stock price data set

The <code>Panel</code> object we created earlier contains much more that then we actually need for this notebook. Thus, we can use a more elegant way to can exaclty the data that we want

In [ ]:
# ask Yahoo! Finance for data and save to DataFrame
data = web.DataReader(tickers, 'yahoo', start, end)['Adj Close']

# save data to a .csv
data.to_csv('./data/raw_data.csv', sep=';')

# show the data
data.head()

In [ ]:
# import data from a .csv
data = pd.read_csv('./data/raw_data.csv', header=0, sep=';', parse_dates=True, index_col=0)

# rearrange data to only show week days
all_weekdays = pd.date_range(start=start, end=end, freq='B')
data = data.reindex(all_weekdays)

# after reindexing we have to amend for missing values
data = data.fillna(method='ffill')
data.head()

How many working days are there in a year?

In [ ]:
# the actual length of one year of working days 365 - 2 * 52
len(pd.date_range(start=datetime.datetime(2012, 5, 18), end=datetime.datetime(2013, 5, 18), freq='B'))

## 1.4 Data visualization

Main tools for plotting in this Notebook are the libraries <code>matplotlib</code>, which contains a lot of the functionalities available in MathLab without the interface, and <code>seaborn</code>.

In [ ]:
# initialize figure and axis
fig, ax = plt.subplots();

# plot resampled series
pas['n_passengers'].plot(ax=ax);

# style the plot
ax.set_title('International Airline Passengers');
ax.set_ylabel('Passengers (in thousands)');
ax.set_xlabel('Year');
ax.xaxis.set_ticks_position('bottom');
fig.tight_layout();

# show the plot
plt.show()

We can see that there must be some kind trend component included in this time series. 

To show this more obiously: By using the <code>resample</code> method on the <code>DataFrame</code> object (our time series) with  argument <code>'AS'</code> we construct a new time series. The series as it is resampled is offest a the beginning of each year. By summing the resampled values we can see a steady increase of total passengers over the years.

In [ ]:
# initialize figure and axis
fig, ax = plt.subplots();

# plot resampled series
pas['n_passengers'].resample('AS').sum().plot(ax=ax)

# style the plot
ax.set_title('International Airline Passengers', fontweight='bold');
ax.set_ylabel('Passengers (in thousands)');
ax.set_xlabel('Year');
ax.xaxis.set_ticks_position('bottom');
fig.tight_layout();

# show the plot
plt.show()

## 1.5 Exercise

Import the earlier created stock prices data set and save it in a formatted fashion to a variable called <code>stock</code>. You may want to use import arguments. It can be found in the <code>./data/raw_data.csv</code> in your corresponding path. Have a look at the top rows and print the summary statistics. 

Asign the Apple stock time series to a <code>Series</code> object named <code>ts</code> and plot it

Open the data folder and have a look at the other data sets. Your are free to explore these as well. Is there anything peculiar about the <code>temp_ger.csv</code> data set?